### TDM을 활용한 지도학습
* 비정형 텍스트 데이터의 전처리를 완료하면 지도학습이 가능하다.

In [1]:
from sklearn.externals import joblib

In [3]:
with open('data/amazon.pkl','rb') as f:
    data = joblib.load(f)
locals().update(data)

In [4]:
tdm

<1000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 4060 stored elements in Compressed Sparse Row format>

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tdm, sentiment, test_size=0.2, 
                                                    random_state=1234)

In [6]:
from sklearn.linear_model import LogisticRegressionCV

In [7]:
model = LogisticRegressionCV()

In [8]:
model.fit(x_train, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [9]:
model.score(x_train, y_train)

0.96250000000000002

In [10]:
model.score(x_test, y_test)

0.78500000000000003

* 감성사전 구축

In [11]:
model.coef_.shape

(1, 1000)

In [12]:
model.coef_[0,:10]

array([ 0.3980101 ,  0.40024896, -0.19642719, -0.34152841,  0.        ,
        0.19202989, -0.33896717,  0.23679641,  0.        ,  0.        ])

* 단어와 회귀계수 짝 만들기

In [14]:
sent_df = pd.DataFrame({'단어': vectorizer.get_feature_names(), '계수':model.coef_.flat})
sent_df.tail()

C:\Users\Hyunsil\Anaconda3\lib\site-packages\pandas\core\dtypes\dtypes.py:271: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


,단어,계수
995,wrongly,-0.251266
996,year,0.047629
997,years,1.441947
998,yell,-0.556230
999,yes,0.000000


* 부정단어사전

In [15]:
neg_df = sent_df[sent_df['계수']<0].sort_values(by='계수')
neg_df.head()

,단어,계수
492,poor,-2.563357
103,bad,-2.239632
803,terrible,-2.128382
988,worst,-1.892567
227,don,-1.884050


* 긍정단어사전

In [16]:
pos_df = sent_df[sent_df['계수']>0].sort_values(by='계수', ascending=False)
pos_df.head()

,단어,계수
310,great,5.266542
985,works,3.297022
114,best,3.264917
395,love,3.105162
255,excellent,3.062604


In [17]:
import numpy as np

* \+ 긍정, - 부정

In [18]:
sent_df['극성'] = np.sign(sent_df['계수'])
sent_df.set_index('단어', inplace=True)
sent_df.tail()

,계수,극성
단어,,
wrongly,-0.251266,-1.0
year,0.047629,1.0
years,1.441947,1.0
yell,-0.556230,-1.0
yes,0.000000,0.0


In [19]:
sentence = 'poort screen but reasonable price'

In [20]:
words = sentence.split()

In [21]:
sent_df.reindex(words)

,계수,극성
단어,,
poort,NaN,NaN
screen,0.175747,1.0
but,NaN,NaN
reasonable,0.297161,1.0
price,2.489535,1.0


In [22]:
sent = sent_df.reindex(words).dropna()
sent

,계수,극성
단어,,
screen,0.175747,1.0
reasonable,0.297161,1.0
price,2.489535,1.0


In [23]:
sent['극성'].sum() / sent['극성'].abs().sum()

1.0